In [ ]:
import torch
import torchvision.transforms as transforms
import os
import pandas as pd
from PIL import Image
import torch.nn as nn
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
from tqdm import tqdm
import torch.optim as optim
import torchvision.transforms.functional as FT
from torch.utils.data import DataLoader

import shutil
import pathlib
import os
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive/
%cd DragonHack


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/My Drive/'
/content/drive/My Drive/DragonHack
[Errno 2] No such file or directory: 'DragonHack'
/content/drive/My Drive/DragonHack


In [ ]:
import os
import csv


def load_annotations(annotation_folder):
    annotations = {}
    for file_name in os.listdir(annotation_folder):
        if file_name.endswith(".txt"):
            image_name = os.path.splitext(file_name)[0] + ".jpg"
            annotations[image_name] = file_name
    return annotations


def write_to_csv(image_folder, annotation_folder, csv_file):
    image_annotations = {}
    for image_file in os.listdir(image_folder):
        if image_file.endswith(".jpg"):
            image_name = image_file
            annotation_file = os.path.splitext(image_file)[0] + ".txt"
            annotation_path = os.path.join(annotation_folder, annotation_file)
            if os.path.exists(annotation_path):
                image_annotations[image_name] = annotation_file

    with open(csv_file, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Image Name", "Annotation"])
        for image_name, annotation_file in image_annotations.items():
            writer.writerow([image_name, annotation_file])


# Path to the folders containing images and annotations
image_folder_path = "images/val"
annotation_folder_path = "labels/val"

# Path to the CSV file
csv_file_path = "val.csv"

# Loading annotations from the annotations folder
annotations = load_annotations(annotation_folder_path)

# Writing data to the CSV file
write_to_csv(image_folder_path, annotation_folder_path, csv_file_path)

print(f"Data has been written to {csv_file_path}")

Data has been written to val.csv


In [ ]:
# Hyperparameters etc.

SEED = 123
LEARNING_RATE = 1e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 16
WEIGHT_DECAY = 0
EPOCHS = 100
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
SAVE_MODEL = True
THRESHOLD = 0.4
IOU_THRESHOLD = 0.5

CLASSES = ["Add"]
C = len(CLASSES)
S = 7

SAVE_MODEL_FILE = "my_checkpoint.pth"
LOAD_MODEL_FILE = "overfit.pth.tar"

TRAIN_DIR = "/train"
TEST_DIR = "/test"
VAL_DIR = "/val"

IMAGE_DIR = "images"
LABEL_DIR = "labels"
PLOT_DIR = "plots"


def normal_transform(image, bboxes):
    transform = transforms.Compose(
        [transforms.Resize((416, 416)), transforms.PILToTensor()]
    )
    image = transform(image).float()
    return image, bboxes

In [ ]:
class YoloDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, img_dir, label_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.transform = transform
        self.S = 7
        self.B = 2
        self.C = C

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        label_path = os.path.join(self.label_dir, self.annotations.iloc[index, 1])
        boxes = []
        c = self.C
        with open(label_path) as f:
            for label in f.readlines():
                class_label, x, y, width, height = [
                    float(x) for x in label.replace("\n", "").split()
                ]
                boxes.append([class_label, x, y, width, height])
            img_path = os.path.join(self.img_dir, self.annotations.iloc[index, 0])
            image = Image.open(img_path)
            boxes = torch.tensor(boxes)

            if self.transform:
                image, boxes = self.transform(image, boxes)

            label_matrix = torch.zeros((self.S, self.S, self.C + 5 * self.B))
            for box in boxes:
                class_label, x, y, width, height = box.tolist()
                class_label = int(class_label)
                i, j = int(self.S * y), int(self.S * x)
                x_cell, y_cell = self.S * x - j, self.S * y - i
                width_cell, height_cell = (width * self.S, height * self.S)
                if label_matrix[i, j, c] == 0:
                    label_matrix[i, j, c] = 1
                    box_coordinates = torch.tensor(
                        [x_cell, y_cell, width_cell, height_cell]
                    )
                    label_matrix[i, j, (c + 1) : (c + 5)] = box_coordinates
                    label_matrix[i, j, class_label] = 1

            return image, label_matrix

In [ ]:
def intersection_over_union(boxes_preds, boxes_labels):
    """
    Calculates intersection over union

    Parameters:
        boxes_preds (tensor): Predictions of Bounding Boxes (BATCH_SIZE, 4) (x, y, w, h)
        boxes_labels (tensor): Correct labels of Bounding Boxes (BATCH_SIZE, 4) (x, y, w, h)

    Returns:
        tensor: Intersection over union for all examples (BATCH_SIZE, 1) (box_iou)
    """

    box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
    box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
    box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
    box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
    box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
    box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
    box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
    box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    # .clamp(0) is for the case when they don't intersect. Since when they don't intersect, one of these will be negative so that should become 0
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)

    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)


def non_max_suppression(bboxes):
    """
    Does Non Max Suppression given bboxes
    Parameters:
        bboxes (list): list of lists containing all bboxes with each bbox
        specified as [class_pred, prob_score, x, y, w, h]
        iou_threshold (float): threshold where predicted bboxes is correct
        threshold (float): threshold to remove predicted bboxes (independent of IoU)
    Returns:
        list: bboxes after performing NMS given a specific IoU threshold
    """

    assert type(bboxes) == list

    threshold = THRESHOLD
    iou_threshold = IOU_THRESHOLD

    bboxes = [box for box in bboxes if box[1] > threshold]
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    bboxes_after_nms = []

    while bboxes:
        chosen_box = bboxes.pop(0)

        bboxes = [
            box
            for box in bboxes
            if box[0] != chosen_box[0]
            or intersection_over_union(
                torch.tensor(chosen_box[2:]),
                torch.tensor(box[2:]),
            )
            < iou_threshold
        ]

        bboxes_after_nms.append(chosen_box)

    return bboxes_after_nms


def get_metrics(loader, model):
    """
    Calculates mean average precision
    Parameters:
        pred_boxes (list): list of lists containing all bboxes with each bboxes
        specified as [train_idx, class_prediction, prob_score, x, y, w, h]
        true_boxes (list): Similar as pred_boxes except all the correct ones
    Returns:
        float: mAP value across all classes given a specific IoU threshold
    """
    pred_boxes, true_boxes = get_bboxes(loader, model)

    iou_threshold = IOU_THRESHOLD
    num_classes = C

    # list storing all AP for respective classes
    average_precisions = []
    f1_scores = []
    classes = []

    # used for numerical stability later on
    epsilon = 1e-6
    true_f1 = 0

    for c in range(num_classes):
        detections = []
        ground_truths = []

        # Go through all predictions and targets,
        # and only add the ones that belong to the
        # current class c
        for detection in pred_boxes:
            if detection[1] == c:
                detections.append(detection)

        for true_box in true_boxes:
            if true_box[1] == c:
                ground_truths.append(true_box)

        # find the amount of bboxes for each training example
        # Counter here finds how many ground truth bboxes we get
        # for each training example, so let's say img 0 has 3,
        # img 1 has 5 then we will obtain a dictionary with:
        # amount_bboxes = {0:3, 1:5}
        amount_bboxes = Counter([gt[0] for gt in ground_truths])

        # We then go through each key, val in this dictionary
        # and convert to the following (w.r.t same example):
        # ammount_bboxes = {0:torch.tensor[0,0,0], 1:torch.tensor[0,0,0,0,0]}
        for key, val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)

        # sort by box probabilities which is index 2
        detections.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))
        total_true_bboxes = len(ground_truths)
        classes.append(total_true_bboxes)

        # If none exists for this class then we can safely skip
        if total_true_bboxes == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            # Only take out the ground_truths that have the same
            # training idx as detection
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]

            best_iou = 0

            for idx, gt in enumerate(ground_truth_img):
                iou = intersection_over_union(
                    torch.tensor(detection[3:]),
                    torch.tensor(gt[3:]),
                )

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                # only detect ground truth detection once
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    # true positive and add this bounding box to seen
                    TP[detection_idx] = 1
                    amount_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1

            # if IOU is lower then the detection is a false positive
            else:
                FP[detection_idx] = 1

        TP_sum = sum(TP.tolist())
        FP_sum = sum(FP.tolist())
        frecall = TP_sum / (total_true_bboxes + epsilon)
        fprecision = TP_sum / (TP_sum + FP_sum + epsilon)
        f1 = 2 * (frecall * fprecision) / (frecall + fprecision + epsilon)
        f1_scores.append(f1)

        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        precisions = torch.divide(TP_cumsum, (TP_cumsum + FP_cumsum + epsilon))
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        # torch.trapz for numerical integration
        average_precisions.append(torch.trapz(precisions, recalls))

    classes = [i / sum(classes) for i in classes]
    for c in range(len(f1_scores)):
        true_f1 += f1_scores[c] * classes[c]

    return sum(average_precisions) / len(average_precisions), true_f1


def get_bboxes(loader, model):

    iou_threshold = IOU_THRESHOLD
    threshold = THRESHOLD
    device = DEVICE

    all_pred_boxes = []
    all_true_boxes = []

    # make sure model is in eval before get bboxes
    model.eval()
    train_idx = 0

    for _, (x, labels) in enumerate(loader):
        x = x.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            predictions = model(x)

        batch_size = x.shape[0]
        true_bboxes = cellboxes_to_boxes(labels)
        bboxes = cellboxes_to_boxes(predictions)

        for idx in range(batch_size):
            nms_boxes = non_max_suppression(bboxes[idx])

            # if batch_idx == 0 and idx == 0:
            #    plot_image(x[idx].permute(1,2,0).to("cpu"), nms_boxes)
            #    print(nms_boxes)

            for nms_box in nms_boxes:
                all_pred_boxes.append([train_idx] + nms_box)

            for box in true_bboxes[idx]:
                # many will get converted to 0 pred
                if box[1] > threshold:
                    all_true_boxes.append([train_idx] + box)

            train_idx += 1

    model.train()
    return all_pred_boxes, all_true_boxes


def convert_cellboxes(predictions):
    """
    Converts bounding boxes output from Yolo with
    an image split size of S into entire image ratios
    rather than relative to cell ratios.
    """

    predictions = predictions.to("cpu")
    batch_size = predictions.shape[0]
    predictions = predictions.reshape(batch_size, S, S, C + 10)
    bboxes1 = predictions[..., C + 1 : C + 5]
    bboxes2 = predictions[..., C + 6 : C + 10]
    scores = torch.cat(
        (predictions[..., C].unsqueeze(0), predictions[..., C + 5].unsqueeze(0)), dim=0
    )
    best_box = scores.argmax(0).unsqueeze(-1)
    best_boxes = bboxes1 * (1 - best_box) + best_box * bboxes2
    cell_indices = torch.arange(7).repeat(batch_size, 7, 1).unsqueeze(-1)
    x = 1 / S * (best_boxes[..., :1] + cell_indices)
    y = 1 / S * (best_boxes[..., 1:2] + cell_indices.permute(0, 2, 1, 3))
    w_y = 1 / S * best_boxes[..., 2:4]
    converted_bboxes = torch.cat((x, y, w_y), dim=-1)
    predicted_class = predictions[..., :C].argmax(-1).unsqueeze(-1)
    best_confidence = torch.max(predictions[..., C], predictions[..., C + 5]).unsqueeze(
        -1
    )
    converted_preds = torch.cat(
        (predicted_class, best_confidence, converted_bboxes), dim=-1
    )

    return converted_preds


def cellboxes_to_boxes(out):
    converted_pred = convert_cellboxes(out).reshape(out.shape[0], S * S, -1)
    converted_pred[..., 0] = converted_pred[..., 0].long()
    all_bboxes = []

    for ex_idx in range(out.shape[0]):
        bboxes = []

        for bbox_idx in range(S * S):
            bboxes.append([x.item() for x in converted_pred[ex_idx, bbox_idx, :]])
        all_bboxes.append(bboxes)

    return all_bboxes


def plot_image(image, boxes):
    """Plots predicted bounding boxes on the image"""

    image = cv2.imread(image)
    im = np.array(image)
    height, width, _ = im.shape

    # Create figure and axes
    _, ax = plt.subplots(1)
    # Display the image
    ax.imshow(im)

    # box[0] is x midpoint, box[2] is width
    # box[1] is y midpoint, box[3] is height

    # Create a Rectangle potch
    for box in boxes:
        box = box[2:]
        assert len(box) == 4, "Got more values than in x, y, w, h, in a box!"
        upper_left_x = box[0] - box[2] / 2
        upper_left_y = box[1] - box[3] / 2
        rect = patches.Rectangle(
            (upper_left_x * width, upper_left_y * height),
            box[2] * width,
            box[3] * height,
            linewidth=1,
            edgecolor="r",
            facecolor="none",
        )
        # Add the patch to the Axes
        ax.add_patch(rect)

    plt.show()


def save_checkpoint(state):
    print("=> Saving checkpoint")
    torch.save(state, SAVE_MODEL_FILE)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [ ]:
class YoloLoss(nn.Module):
    def __init__(self):
        super(YoloLoss, self).__init__()
        self.mse = nn.MSELoss(reduction="sum")
        self.S = 7
        self.B = 2
        self.C = C
        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        predictions = predictions.reshape(-1, self.S, self.S, self.C + self.B * 5)
        iou_b1 = intersection_over_union(
            predictions[..., self.C + 1 : self.C + 5],
            target[..., self.C + 1 : self.C + 5],
        )
        iou_b2 = intersection_over_union(
            predictions[..., self.C + 6 : self.C + 10],
            target[..., self.C + 1 : self.C + 5],
        )
        ious = torch.cat([iou_b1.unsqueeze(0), iou_b2.unsqueeze(0)], dim=0)
        _, best_box = torch.max(ious, dim=0)
        exists_box = target[..., self.C].unsqueeze(3)  # Iobj_i

        # ========================= #
        #    FOR BOX COORDINATES    #
        # ========================= #

        box_predictions = exists_box * (
            best_box * predictions[..., self.C + 6 : self.C + 10]
            + (1 - best_box) * predictions[..., self.C + 1 : self.C + 5]
        )
        box_targets = exists_box * target[..., self.C + 1 : self.C + 5]
        box_predictions[..., 2:4] = torch.sign(box_predictions[..., 2:4]) * torch.sqrt(
            torch.abs(box_predictions[..., 2:4] + 1e-6)
        )

        box_targets[..., 2:4] = torch.sqrt(box_targets[..., 2:4])

        # (N, S, S, 4) -> (N*S*S, 4)
        box_loss = self.mse(
            torch.flatten(box_predictions, end_dim=-2),
            torch.flatten(box_targets, end_dim=-2),
        )

        # ===================== #
        #    FOR OBJECT LOSS    #
        # ===================== #

        pred_box = (
            best_box * predictions[..., self.C + 5 : self.C + 6]
            + (1 - best_box) * predictions[..., self.C : self.C + 1]
        )

        # (N*S*S, 1)
        object_loss = self.mse(
            torch.flatten(exists_box * pred_box),
            torch.flatten(exists_box * target[..., self.C : self.C + 1]),
        )

        # ======================== #
        #    FOR NO OBJECT LOSS    #
        # ======================== #

        # (N, S, S, 1) --> (N, S*S)
        no_object_loss = self.mse(
            torch.flatten(
                (1 - exists_box) * predictions[..., self.C : self.C + 1], start_dim=1
            ),
            torch.flatten(
                (1 - exists_box) * target[..., self.C : self.C + 1], start_dim=1
            ),
        )
        no_object_loss += self.mse(
            torch.flatten(
                (1 - exists_box) * predictions[..., self.C + 5 : self.C + 6],
                start_dim=1,
            ),
            torch.flatten(
                (1 - exists_box) * target[..., self.C : self.C + 1], start_dim=1
            ),
        )

        # ==================== #
        #    FOR CLASS LOSS    #
        # ==================== #

        # (N, S, S, 20) -> (N*S*S, 20)
        class_loss = self.mse(
            torch.flatten(exists_box * predictions[..., : self.C], end_dim=-2),
            torch.flatten(exists_box * target[..., : self.C], end_dim=-2),
        )

        loss = (
            self.lambda_coord * box_loss
            + object_loss
            + self.lambda_noobj * no_object_loss
            + class_loss
        )

        return loss

In [ ]:
architecture_config = [
    # Tuple: (kernel_size, number of filters, strides, padding)
    # "M" = Max Pool Layer
    # List: [(tuple), (tuple), how many times to repeat]
    # Doesnt include fc layers
    (7, 64, 2, 3),
    "M",
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],
    (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1),
]


class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(CNNBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.leakyrelu = nn.LeakyReLU(0.1)

    def forward(self, x):
        return self.leakyrelu(self.conv(x))


class YoloV1(nn.Module):
    def __init__(self):
        super(YoloV1, self).__init__()
        self.architecture = architecture_config
        self.in_channels = 3
        self.darknet = self._create_conv_layers(self.architecture)
        self.fcs = self._create_fcs()

    def forward(self, x):
        x = self.darknet(x)
        return self.fcs(torch.flatten(x, start_dim=1))

    def _create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == tuple:
                layers += [
                    CNNBlock(
                        in_channels, x[1], kernel_size=x[0], stride=x[2], padding=x[3]
                    )
                ]
                in_channels = x[1]
            elif type(x) == str:
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            elif type(x) == list:
                conv1 = x[0]  # Tuple
                conv2 = x[1]  # Tuple
                repeats = x[2]  # Int

                for _ in range(repeats):
                    layers += [
                        CNNBlock(
                            in_channels,
                            conv1[1],
                            kernel_size=conv1[0],
                            stride=conv1[2],
                            padding=conv1[3],
                        )
                    ]
                    layers += [
                        CNNBlock(
                            conv1[1],
                            conv2[1],
                            kernel_size=conv2[0],
                            stride=conv2[2],
                            padding=conv2[3],
                        )
                    ]
                    in_channels = conv2[1]

        return nn.Sequential(*layers)

    def _create_fcs(self):
        B = 2
        return nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * S * S, 496),
            nn.Dropout(0.0),
            nn.LeakyReLU(0.1),
            nn.Linear(496, S * S * (C + B * 5)),
        )

In [ ]:
torch.manual_seed(SEED)


def train_fn(train_loader, model, optimizer, loss_fn):
    loop = tqdm(train_loader, leave=True)
    mean_loss = []

    for _, (x, y) in enumerate(loop):
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update the progress bar
        loop.set_postfix(loss=loss.item())
    avg = sum(mean_loss) / len(mean_loss)
    print(f"Mean loss was {avg}")

    return avg


def main():
    model = YoloV1().to(DEVICE)
    optimizer = optim.Adam(
        model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )
    loss_fn = YoloLoss()
    transform = normal_transform

    if LOAD_MODEL:
        load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

    train_dataset = YoloDataset(
        "train.csv",
        transform=transform,
        img_dir=IMAGE_DIR + TRAIN_DIR,
        label_dir=LABEL_DIR + TRAIN_DIR,
    )

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )
    val_dataset = YoloDataset(
        "val.csv",
        transform=transform,
        img_dir=IMAGE_DIR + TRAIN_DIR,
        label_dir=LABEL_DIR + TRAIN_DIR,
    )

    val_loader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )
    best_f1 = 0
    big_loss = 10000
    fail_counter = 0
    for epoch in range(EPOCHS):

        print(f"On epoch: {epoch + 1}")
        if epoch > 9 and epoch % 5 == 0:
            mean_avg_prec, f1 = get_metrics(val_loader, model)
            print(f"Train mAP: {mean_avg_prec}")
            print(f"Train F1 score: {f1}")

        current_loss = train_fn(train_loader, model, optimizer, loss_fn)

        if current_loss < big_loss:
            big_loss = current_loss
            fail_counter = 0
        else:
            fail_counter += 1

        # if fail_counter > 3:
        #  for g in optimizer.param_groups:
        #  g['lr'] *= 2

        # if epoch % 10 == 0:
        # for g in optimizer.param_groups:
        #  g['lr'] /= 2

        if epoch > 50 and f1 > best_f1:
            best_f1 = f1
            checkpoint = {
                "state_dict": model.state_dict(),
                "optimizer": optimizer.state_dict(),
            }
            save_checkpoint(checkpoint)


if __name__ == "__main__":
    main()

On epoch: 1


  0%|          | 0/41 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 41/41 [00:12<00:00,  3.61it/s, loss=188]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 41/41 [00:12<00:00,  3.18it/s, loss=188]


Mean loss was 364.03588941620615
On epoch: 2


100%|██████████| 41/41 [00:13<00:00,  3.12it/s, loss=132]


Mean loss was 130.9339034848097
On epoch: 3


...

Mean loss was 8.895252425496171
On epoch: 99


100%|██████████| 41/41 [00:13<00:00,  3.00it/s, loss=9.63]


Mean loss was 8.61039879263901
On epoch: 100


100%|██████████| 41/41 [00:13<00:00,  3.07it/s, loss=9.33]

Mean loss was 8.42773009509575
